In [3]:
from tqdm import tqdm
# import random
import os 
import pydub
from pydub import AudioSegment
import numpy as np
from pathlib import Path
import pickle

In [52]:
labels = ['Alcedo atthis', 'Botaurus stellaris', 'Gallinula chloropus', 'Motacilla flava', 'Tachybaptus ruficollis']
labels1 = ['Alcedo_atthis', 'Botaurus_stellaris', 'Gallinula_chloropus', 'Motacilla_flava', 'Tachybaptus_ruficollis']

In [54]:
def read(f, normalized=False):
    """MP3 to numpy array"""
    a = pydub.AudioSegment.from_mp3(f)
    y = np.array(a.get_array_of_samples())
    if a.channels == 2:
        y = y.reshape((-1, 2))
    if normalized:
        return a.frame_rate, np.float32(y) / 2**15
    else:
        return a.frame_rate, y

birds = []
for label in labels:
    for i in range(1, 41):
        birds.append(read(f'raw_WMWBD/{label}/b ({i}).mp3'))

In [57]:
y = []
for i in range(200):
    y.append(i // 40)

In [60]:
data = birds, y
with open('WMWBD.data', 'wb') as f:
        pickle.dump(data, f)

In [4]:
with open('WMWBD.data', 'rb') as f:
        data_x, data_y = pickle.load(f)

In [8]:
for k in range(5):
    a1 = 0
    a2 = 0
    for i in range(40):
        if data_x[i][0] == 44100:
            a1 += 1
        if data_x[i][0] == 48000:
            a2 += 1
    print(a1, a2)

19 21
19 21
19 21
19 21
19 21


In [37]:
x_len = 88000

ds_x = []
ds_y = []
for k in range(5):
    for i in range(40):
        if data_x[i][0] == 48000:
            slice = (len(data_x[i][1]) - x_len) // 2
            if (len(data_x[i][1]) - x_len) % 2 == 0:
                offset = 0 
            else:
                offset = 1
            ds_x.append(data_x[i][1][slice + offset: -slice])
            ds_y.append(i)

In [39]:
data = ds_x, ds_y
with open('WMWBD_small.data', 'wb') as f:
        pickle.dump(data, f)